# Cliquet
Price a Cliquet using different models. Cliquets are frequently used as hedges for annuities with caps and floors.
For a definition of the payoff, and the dataclass to construct a cliquet timetable, see [qablet_contracts.eq.cliquet.Accumulator](https://qablet.github.io/qablet-contracts/examples/equity_cliquet)

In [ ]:
from datetime import datetime

import pandas as pd
from qablet.black_scholes.mc import LVMCModel
from qablet.heston.mc import HestonMCModel
from qablet_contracts.eq.cliquet import Accumulator

from data.samples import heston_data, localvol_data, rbergomi_data
from src.model.rbergomi import rBergomiMCModel

### Contract
Create a Cliquet Contract fixing bi-monthly.

In [ ]:
fix_dates = pd.bdate_range(
    datetime(2013, 8, 14), datetime(2014, 12, 31), freq="2BME"
)
global_floor = 0.0
local_floor = -0.03
local_cap = 0.05
timetable = Accumulator(
    "USD",
    "SPX",
    fix_dates,
    global_floor,
    local_floor,
    local_cap,
    state={"S_PREV": 1.0},
).timetable()

print(timetable["events"].to_pandas())

   track                      time   op  quantity     unit
0    NaN 2013-08-30 00:00:00+00:00  NaN       0.0     INIT
1    NaN 2013-10-31 00:00:00+00:00  NaN       0.0  CALCFIX
2    NaN 2013-12-31 00:00:00+00:00  NaN       0.0  CALCFIX
3    NaN 2014-02-28 00:00:00+00:00  NaN       0.0  CALCFIX
4    NaN 2014-04-30 00:00:00+00:00  NaN       0.0  CALCFIX
5    NaN 2014-06-30 00:00:00+00:00  NaN       0.0  CALCFIX
6    NaN 2014-08-29 00:00:00+00:00  NaN       0.0  CALCFIX
7    NaN 2014-10-31 00:00:00+00:00  NaN       0.0  CALCFIX
8    NaN 2014-12-31 00:00:00+00:00  NaN       0.0  CALCFIX
9        2014-12-31 00:00:00+00:00    >       0.0      USD
10       2014-12-31 00:00:00+00:00    +       1.0      ACC


### Compare Models
Compare different models, starting with Local Volatility Model.

In [ ]:
model = LVMCModel()
price, _ = model.price(timetable, localvol_data())
print(f"Localvol price: {price}")

Localvol price: 0.09835745819114465


Heston Model

In [ ]:
model = HestonMCModel()
price, _ = model.price(timetable, heston_data())
print(f"Heston price: {price}")

Heston price: 0.07816623272486052


Bergomi Model

In [ ]:
model = rBergomiMCModel()
dataset = rbergomi_data()
dataset["MC"]["PATHS"] = 20_000  # very slow with 100_000, 1/250
dataset["MC"]["TIMESTEP"] = 1 / 100
price, _ = model.price(timetable, dataset)
print(f"rBergomi price: {price}")

rBergomi price: 0.08386114469064239
